In [2]:
import json
import pandas as pd
from urllib.request import Request, urlopen

### Browse Schools
#### This API request returns a single school or a list of K-12 schools (and related directory & GreatSchools 1-10 Rating data) based on the parameters passed to it.

In [3]:
def api_puller(zipcode):
    """
    The api_puller function accepts the zipcode input (as a string) and returns a dictionary of the schools located in that zipcode

    :param zipcode:
    :return: export_json
    """
    apikey = 'lB6mbjJnVv1S307NPl4B61pPfDXH9Gxn5QxpdTQu'
    req = Request('https://gs-api.greatschools.org/schools?zip=' + zipcode)
    req.add_header('X-API-Key', apikey)
    content = urlopen(req).read()
    export_json = json.loads(content)
    return export_json

In [4]:
zipCodes = pd.read_csv('zipCodes.txt')

In [8]:
zipInput = '22214'
dataTest = api_puller(zipInput)

In [16]:
maxReq = 900
for zip in zipCodes.ZipCode.head(maxReq):
    zipStr = str(zip)
    dataGrab = api_puller(zipStr)
    zipSchools[zipStr] = dataGrab

#### Loads json data to zipSchools dictionary

In [13]:
with open('schoolData.json', 'r') as openfile:
    zipSchools = json.load(openfile)

#### Saves zipSchools dictionary to schoolData.json

In [17]:
with open("schoolData.json", "w") as outfile:
    json.dump(zipSchools, outfile)

In [53]:
columns = ['universalID', 'name', 'type', 'level-code', 'zip', 'rating']

schools_df = pd.DataFrame(columns=columns)

In [56]:
allIDs = {}
schools_df = pd.DataFrame(columns=columns)
for zip in zipSchools:
    for entry in zipSchools[zip]['schools']:
        universalID = entry['universal-id']
        name = entry['name']
        type = entry['type']
        levelCode = entry['level-codes']
        zip = entry['zip']
        rating = entry['rating']
        allIDs[universalID] = [name, type, levelCode, zip, rating]


print(len(allIDs))
schools_df = pd.DataFrame(allIDs)
newSchools_df = schools_df.transpose()
newSchools_df.rename(['Name', 'Type', 'Level', 'Zip', 'Rating'], inplace=True)

4968


In [58]:
newSchools_df.rename(columns={0: 'Name', 1: 'Type', 2: 'Level', 3: 'Zip', 4: 'Rating'}, inplace=True)

In [62]:
newSchools_df.to_csv('greatSchools.csv')

## Integration into GeoPandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd ###
from shapely.geometry import Point, Polygon###
from fiona.drvsupport import supported_drivers###
import folium as fm
import re
%matplotlib inline

#Crime dataframe
crime_df = pd.read_csv('2020_FBI_Crime_Statistics.csv')
crime_df = crime_df.fillna(0) #fill in Nan with 0's
crime_df['Crime Index'].astype('float64')

#Zip Code data frame
supported_drivers['KML'] = 'rw'
place_df = gpd.read_file('./Geographies/DC_MD_VA_Zipcodes.kml', driver='KML')

def find_city_name(cell):
    match = re.search(r'<at><openparen>([A-Za-z ]+)<closeparen>', cell)
    if match:
        return match.group(1)
    else:
        return None

cities = list()
for r in place_df.iterrows():
    cities.append(find_city_name(r[1]['Name']))

place_df['City'] = cities

#Merging crime and zip DataFrames. Not that not every city in zip has reported crime data to FBI data base
crime_df_merge = pd.merge(place_df, crime_df[['City','Crime Index']], on='City', how = 'left')

crime_df_merge = crime_df_merge.fillna(0) #filling Nan with 0's

#setting cities without reported crime to the average of the population reported by FBI
crime_df_merge.loc[crime_df_merge['Crime Index']==0, ['Crime Index']] = 4.46
display(crime_df_merge.head())

### Nearby Schools
#### This API request returns a list of K-12 schools (and related directory & GreatSchools 1-10 Rating data) closest to an identified location.
##### Not built out because we are using the "Browse Schools" API Call

In [ ]:
nearbyReq = Request('https://gs-api.greatschools.org/nearby-schools?lat=37.7940627&lon=-122.2680029&limit=3&distance=5')
nearbyReq.add_header('X-API-Key', apiKey)
nearbyContent = urlopen(nearbyReq).read()

## Check API Key Status

In [14]:
def APIStatus():
    apikey = 'lB6mbjJnVv1S307NPl4B61pPfDXH9Gxn5QxpdTQu'
    APIreq = Request('https://gs-api.greatschools.org/usage?api_key=' + apikey)
    APIreq.add_header('X-API-Key', apikey)
    APIcontent = urlopen(APIreq).read()
    exportAPI = json.loads(APIcontent)
    return exportAPI

In [54]:
APIStatus()

{'calls_made': 913,
 'remaining_quota': 87,
 'time_period': '2023-03-01 to 2023-03-31'}